In [ ]:
# !pip install requests
# !pip install beautifulsoup4

**뉴스 크롤링 함수**
<br> <br>
- 시황, 전망 카테고리 : 401
- 기업, 종목분석 : 402
- 해외증시 : 403
- 채권선물 : 404
- 환율 : 429

In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import html
import requests
from bs4 import BeautifulSoup
from urllib.parse import unquote

import pandas as pd
from datetime import datetime, timedelta

In [ ]:
def get_start_target_dates(date, n:int):
    '''
    Get the start and target dates for crawling based on the specified date
    data : 내가 확인하고 싶은 날짜, n : 며칠 전까지 확인하고 싶은지 입력
    '''

    from datetime import datetime, timedelta

    target_date = datetime.strptime(date, '%Y%m%d')
    
    # Crawling 3 days before the target date (예: 금,토,일,월)
    start_date = target_date - timedelta(days=n)  

    # Convert start_date and target_date back to string format
    start_date_str = start_date.strftime('%Y%m%d')
    target_date_str = target_date.strftime('%Y%m%d')

    return start_date_str, target_date_str

In [ ]:
def News_crawling(id_num, date):

    ''' 네이버 증권 뉴스 "시황.전망" 뉴스 기사 크롤링 함수 '''

    # 웹 페이지 URL 설정
    url = 'https://finance.naver.com/news/news_list.naver?mode=LSS3D&section_id=101&section_id2=258&section_id3=' + id_num + '&date=' + date + '&page={}'
    print('url:', url)

    # ====================================================================================================
    
    # 맨 마지막 페이지 추출
    last_page_number = 1
    while True:
        response = requests.get(url.format(last_page_number))
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            # 페이지가 존재하지 않는 경우 조건 추가
            if not soup.select('dt.articleSubject, dd.articleSubject'):
                last_page_number -= 1
                break
            last_page_number += 1
        else:
            # 404 에러 발생 시, 직전 페이지가 맨 마지막 페이지임
            last_page_number -= 1
            break

    print("맨 마지막 페이지:", last_page_number)

    # ====================================================================================================

    # Create lists to store the data
    titles = []
    news_sources = []
    dates = []
    main_contents = []
    decoded_hrefs = []

    # 모든 페이지의 데이터 크롤링
    for page in range(1, last_page_number + 1):
        response = requests.get(url.format(page))
        encoding = response.encoding  # 웹 페이지의 인코딩 확인
        soup = BeautifulSoup(response.content, 'html.parser', from_encoding=encoding)

        if response.status_code == 200:
            # <dt class="articleSubject"> / <dd class="articleSubject"> 태그들을 가져오기
            article_subject_elements = soup.select('dt.articleSubject, dd.articleSubject')

            # 제목과 관련된 href 가져오기
            for article_subject_element in article_subject_elements:
                title = article_subject_element.text
                a_tag = article_subject_element.find('a')
                raw_href = a_tag['href']
                decoded_href = raw_href.replace('§', '&sect')  # § 문자를 &sect로 변환
                decoded_href = 'https://finance.naver.com' + decoded_href

                # ====================================================================================================

                # 뉴스 본문 페이지로 다시 요청
                news_response = requests.get(decoded_href)
                news_soup = BeautifulSoup(news_response.content, 'html.parser', from_encoding=news_response.encoding)

                # 뉴스 제목 가져오기
                news_source_tag = news_soup.select_one('span.press')
                news_source = news_source_tag.img['title'] if news_source_tag and news_source_tag.img else '출처 정보 없음'

                # 날짜 가져오기
                date_tag = news_soup.select_one('span.article_date')
                date = date_tag.text.strip() if date_tag else '날짜 정보 없음'

                # 본문 내용 가져오기
                main_content = news_soup.select_one('div.articleCont#content')

                if main_content:
                    # <em class="img_desc"> 태그 안의 내용은 제외하고 추출
                    unwanted_tag = main_content.find('em', class_='img_desc')
                    if unwanted_tag:
                        unwanted_tag.extract()  # 특정 태그 제외

                    # <strong class="media_end_summary"> 태그와 내용 제외하기
                    strong_tag = main_content.find('strong', class_='media_end_summary')
                    if strong_tag:
                        strong_tag.extract()  # 특정 태그 제외

                # Append the data to the lists
                titles.append(title)
                news_sources.append(news_source)
                dates.append(date)
                main_contents.append(main_content.text.strip())
                decoded_hrefs.append(decoded_href)

        else:
            print(f"페이지 {page}에 접근할 수 없습니다.")

        # ====================================================================================================

    # Create a DataFrame from the lists
    data = {
        '제목': titles,
        '뉴스 출처': news_sources,
        '날짜': dates,
        '뉴스 본문': main_contents,
        '관련 href': decoded_hrefs
    }

    df = pd.DataFrame(data)
    df.to_csv(f'/content/drive/MyDrive/Colab Notebooks/{date}_category{id_num}_news_크롤링.csv', index=False, encoding='utf-8')

    return df

In [ ]:
# 확인하고 싶은 날짜 입력
start_date, target_date = get_start_target_dates('20230808', 1)           
date_range = pd.date_range(start=start_date, end=target_date, freq='D')

temp_lst = []
for date in date_range:
    date_str = date.strftime('%Y%m%d')
    temp_df = News_crawling('429', date_str)
    temp_lst.append(temp_df)

temp_final = pd.concat(temp_lst)

url: https://finance.naver.com/news/news_list.naver?mode=LSS3D&section_id=101&section_id2=258&section_id3=429&date=20230807&page={}
맨 마지막 페이지: 1
url: https://finance.naver.com/news/news_list.naver?mode=LSS3D&section_id=101&section_id2=258&section_id3=429&date=20230808&page={}
맨 마지막 페이지: 1


In [ ]:
temp_final

,제목,뉴스 출처,날짜,뉴스 본문,관련 href
0,"\n원/달러 환율, 미국CPI 발표 앞두고 1,300원대서 하락 마감\n",연합뉴스,2023-08-07 15:49,(서울=연합뉴스) 오지은 기자 = 이번 주에 미국 7월 소비자물가지수(CPI) 발표...,https://finance.naver.com/news/news_read.naver...
1,"\n코스피, 2,600 내외서 나흘째 약세…이차전지주 동반 하락(종합)\n",연합뉴스,2023-08-07 09:33,(서울=연합뉴스) 홍유담 기자 = 코스피가 7일 장 초반 투자자들의 관망세에 나흘째...,https://finance.naver.com/news/news_read.naver...
2,"\n원/달러 환율, 미국 고용지표 둔화 속 1,300원대 하락 출발\n",연합뉴스,2023-08-07 09:25,(서울=연합뉴스) 오지은 기자 = 지난주 30원가량 급등했던 원/달러 환율이 미국 ...,https://finance.naver.com/news/news_read.naver...
3,\n금리 뛰자 장롱속 신사임당 돌아왔다…5만원권 환수율 역대 최고\n,연합뉴스,2023-08-06 06:05,상반기 5만원권 환수율 77.8%…발행액 10조원·환수액 7.8조원사회적 거리 두기...,https://finance.naver.com/news/news_read.naver...
4,\n[그래픽] 원/달러 환율 추이\n,연합뉴스,2023-08-04 16:20,(서울=연합뉴스) 원형민 기자 = 4일 서울 외환시장에서 달러 대비 원화 환율은 전...,https://finance.naver.com/news/news_read.naver...
5,"\n원/달러 환율, 10원 넘게 급등…1,310원 근접 마감\n",연합뉴스,2023-08-04 15:48,"(서울=연합뉴스) 채새롬 기자 = 원/달러 환율이 4일 장중 10원 넘게 올라 1,...",https://finance.naver.com/news/news_read.naver...
6,"\n원/달러 환율, 미국 고용지표 대기…장 초반 보합권\n",연합뉴스,2023-08-04 09:24,(서울=연합뉴스) 채새롬 기자 = 4일 원/달러 환율은 미국 고용지표 발표를 대기하...,https://finance.naver.com/news/news_read.naver...
7,"\n원/달러 환율, 3주 만에 1,300원선 '터치'\n",연합뉴스,2023-08-03 15:41,"(서울=연합뉴스) 채새롬 기자 = 3일 원/달러 환율이 1,300원선을 찍고 내려왔...",https://finance.naver.com/news/news_read.naver...
8,"\n일본, 금융완화 수정에도 엔저 지속…국채 수익률은 상승세\n",연합뉴스,2023-08-03 15:22,(도쿄=연합뉴스) 박상현 특파원 = 일본 중앙은행인 일본은행이 7개월 만에 대규모 ...,https://finance.naver.com/news/news_read.naver...
9,"\n[게시판] KB국민은행, 거래외국환은행 지정 고객에 1만원 쿠폰\n",연합뉴스,2023-08-03 15:15,▲ KB국민은행은 다음 달 29일까지 유학생과 해외체재 고객을 대상으로 KB금융쿠폰...,https://finance.naver.com/news/news_read.naver...


In [ ]:
# temp_final.to_csv(f'/content/drive/MyDrive/Colab Notebooks/시황전망_news_크롤링.csv', index=False, encoding='utf-8')
# temp_final.to_csv(f'/content/drive/MyDrive/Colab Notebooks/기업종목분석_news_크롤링.csv', index=False, encoding='utf-8')
# temp_final.to_csv(f'/content/drive/MyDrive/Colab Notebooks/해외증시_news_크롤링.csv', index=False, encoding='utf-8')
# temp_final.to_csv(f'/content/drive/MyDrive/Colab Notebooks/채권선물_news_크롤링.csv', index=False, encoding='utf-8')
# temp_final.to_csv(f'/content/drive/MyDrive/Colab Notebooks/환율_news_크롤링.csv', index=False, encoding='utf-8')